## Experiments with time series data

In [ ]:
import yaml

import tabmemcheck

import statutils
import tabular_queries

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import pickle

In [ ]:
experiments = {
        "netflix": [2020, 2022],
        "nasdaqcomp": [2020, 2022],
        "msci-world": [2020, 2022],
        "usd-eur": [2020, 2022],
        "usd-yuan": [2020, 2022],
}

In [ ]:
results = {}
for time_series_name, years in experiments.items():
    for year in years:
        # the experiment
        exp_name = time_series_name + "-" + str(year)

        # load the csv file
        csv_file = f"datasets/csv/time-series/{exp_name}.csv"
        df = pd.read_csv(csv_file)

        # the names of the features and the target
        feature_names, target_name = (
            df.columns.tolist()[:-1],
            df.columns.tolist()[-1],
        )

        # the data
        X_data, y_data = df[feature_names].values, df[target_name].values

        # chatlogs
        messages, responses = tabmemcheck.read_chatlog(exp_name, root="../GPT4-experiments/gpt-4-chatlogs/")

        T_max = df.shape[0]
        N = T_max -20

        labels = [y_data[idx] for idx in range(20, T_max)]
        predictions = [tabular_queries.read_float(x["choices"][0]["message"]["content"], default=-100) for x in responses]
        tokens =  [x["usage"]["completion_tokens"] for x in responses]

        # too many tokens = invalid response
        invalid_idx = np.where(tokens > 2 * np.mean(tokens))[0].tolist()
        invalid_idx.extend(np.where(np.array(predictions) == -100)[0].tolist())
        print('invalid and replaced indices: ', invalid_idx)
        for idx in invalid_idx: # replace with the value of the previous day
            predictions[idx] = y_data[20 + idx - 1]

        # BASELINE: predict the value of the previous day for the current day
        predictions = [y_data[20 + idx - 1] for idx in range(N)]

        # store results
        results[exp_name] = (predictions, labels)

        # plot labels and predictions 
        """
        import matplotlib.pyplot as plt
        plt.plot(list(range(N)), labels, label="labels")
        plt.plot(list(range(N)), predictions, label="predictions")
        plt.title(exp_name)
        plt.legend()
        plt.show()
        

        # at every point, compute the relative error of the prediction versus the label
        predictions = np.array(predictions)
        labels = np.array(labels)

        # exclude areas where the label is zero
        valid_indices = np.where(labels != 0)[0]
        if len(valid_indices) < len(labels):
            print(f'excluded {len(labels)-len(valid_indices)} timesteps where the time series is zero')
        predictions = predictions[valid_indices]
        labels = labels[valid_indices]

        rel_error = np.abs((predictions - labels) / labels)
        # introduce a ceiling of one 
        print(np.mean(rel_error) * 100, np.std(rel_error) * 100 / np.sqrt(len(rel_error)))
        """

### yuan - usd in 2015

In [ ]:
# adjust font size in the plots
plt.rcParams.update({'font.size': 13})

# figure size
plt.rcParams['figure.figsize'] = [7, 5]

# plot the august of 2015 yuan versus usd exchange rate
predictions, labels = results['usd-yuan-2015']
predictions = np.array(predictions)
labels = np.array(labels)

# load the original data to get the dates for the labels
csv_file = f"datasets/csv/time-series/usd-yuan-2015.csv"
df = pd.read_csv(csv_file)

# the names of the features and the target
dates = df['Date'].values
dates = [dates[idx] for idx in range(20, T_max)]
august_indices = [idx for idx in range(len(dates)) if dates[idx][5:7] == '08']

# plot the august, using dotted lines for the predictions and solid lines for the labels
plt.plot(list(range(len(august_indices))), labels[august_indices], '.-', label='USD-Yuan Exchange Rate', lw=1, ms=15)
plt.plot(list(range(len(august_indices))), predictions[august_indices], '.-', label='GTP-3.5 Prediction', lw=1, ms=15)
# vertical line at 6
#plt.axvline(x=6, color='r', linestyle='--')
plt.xlabel('August 2015')
plt.ylabel('USD-Yuan Exchange Rate')
# set position of xtick labels
plt.gca().set_xticks([0, 6, 20])
labels = [item.get_text() for item in plt.gca().get_xticklabels()]
labels[0] = '3rd August'
labels[1] = '11th August'
labels[2] = '31st August'
plt.gca().set_xticklabels(labels)
plt.legend()
plt.show()

#### compute the relative error

In [ ]:
for exp_name, v in results.items():
    predictions, labels = v
    
    # at every point, compute the relative error of the prediction versus the label
    predictions = np.array(predictions)
    labels = np.array(labels)

    # compute the relative error
    rel_error = np.abs((predictions - labels) / (labels+1e-14))

    # introduce a ceiling of of 0.02
    rel_error = np.minimum(rel_error, 0.02) 

    # print result
    print(f'{exp_name}: {np.nanmean(rel_error) * 100:.2f} ({np.nanstd(rel_error) * 100 / np.sqrt(len(rel_error)):.2f})')

#### ability to predict the sign of the change (better than random before 2022)

In [ ]:
for exp_name, v in results.items():
    predictions, labels = v
    
    # at every point, compute the relative error of the prediction versus the label
    predictions = np.array(predictions)
    labels = np.array(labels)

    # compute the change from the previous day
    predictions_diff = predictions[1:] - predictions[:-1]
    labels_diff = labels[1:] - labels[:-1]

    # was the sign predicted correctly?
    outcome = np.sign(predictions_diff) == np.sign(labels_diff)

    # introduce a ceiling of one
    print(f'{exp_name}: {np.nanmean(outcome) * 100:.2f} ({np.nanstd(outcome) * 100 / np.sqrt(len(outcome)):.2f})')

# Weather data

In [ ]:
cities = ['berlin', 'tokyo', 'jakarta', 'moscow', 'singapore', 'washington', 'london', 'paris', 'cairo', 'vancouver']
cities.sort()

for city in cities:
    latex_table_row = ''

    # 2015-2019
    means = []
    for year in [2015, 2016, 2017, 2018, 2019]:
        predictions, labels = results[city + '-temperature-' + str(year)]
        predictions, labels = np.array(predictions), np.array(labels)
        rel_error = np.abs((predictions - labels) / (labels+1e-14))
        rel_error = np.minimum(rel_error, 1) # max error of 100% at every time step
        mean_rel_error = np.nanmean(rel_error) * 100
        means.append(mean_rel_error)
    latex_table_row = city + f" & {min(means):.2f}\\% - {max(means):.2f}\\% "

    # 2020
    for year in [2020, 2022]:
        predictions, labels = results[city + '-temperature-' + str(year)]
        predictions, labels = np.array(predictions), np.array(labels)
        rel_error = np.abs((predictions - labels) / (labels+1e-14))
        rel_error = np.minimum(rel_error, 1) # max error of 100% at every time step
        mean_rel_error = np.nanmean(rel_error) * 100
        std_rel_error = np.nanstd(rel_error) * 100 / np.sqrt(len(rel_error))
        latex_table_row += f'& {mean_rel_error:.2f}\\% '

    latex_table_row += ' \\\\'
    print(latex_table_row)

In [ ]:
# weather index
for year in [2015, 2016, 2017, 2018, 2019, 2020, 2022]:
    composite_index_value = []
    for city in cities:
        predictions, labels = results[city + '-temperature-' + str(year)]
        predictions, labels = np.array(predictions), np.array(labels)
        rel_error = np.abs((predictions - labels) / (labels+1e-14))
        rel_error = np.minimum(rel_error, 1) # max error of 100% at every time step
        mean_rel_error = np.nanmean(rel_error) * 100
        composite_index_value.append(mean_rel_error)
    print(year, f'{np.mean(composite_index_value):.2f}', f'{np.std(composite_index_value) / np.sqrt(len(composite_index_value)):.2f}')